In [ ]:
import pandas as pd
import os
import requests
from datetime import datetime, timedelta, date
import io
import yfinance as yf
import openpyxl

Termsheet = pd.read_csv("C:\\Users\\dbcin\\OneDrive\\Desktop\\Equity File Storage\\EQUITY_L (1).csv")
df = pd.DataFrame(Termsheet)
filter_condition = df[' SERIES'] == "EQ"
Data = df[filter_condition]
ticker_symbols = Data.SYMBOL

base_folder = r"C:\Users\dbcin\OneDrive\Desktop\Equity File Storage"
folder_name = datetime.today().strftime('%d-%B-%Y')
folder_path = os.path.join(base_folder, folder_name)

#print(f"Folder path reads as follows: '{folder_path}'")

if not os.path.exists(folder_path):
    os.makedirs(folder_path, exist_ok= True)
    print(f"Folder '{folder_name}' created.")
else:
    print(f"Folder '{folder_name}' already exists.")

In [ ]:
# Calculate start and end dates
end_date_str = (datetime.today() + timedelta(days=1)).strftime('%d-%m-%Y')
start_date_str = (datetime.today() - timedelta(days=500)).strftime('%d-%m-%Y')
print(f"Today's Date is {end_date_str}. The t-period commences on {start_date_str}")

#Converting start and end date to datetime objects
end_date = datetime.strptime(end_date_str,'%d-%m-%Y')
start_date = datetime.strptime(start_date_str,'%d-%m-%Y')

In [ ]:
error_tickers = []
total_files_downloaded = 0

for ticker in ticker_symbols:
    try:
        stockdata = yf.download(f'{ticker}.NS', start = start_date, end = end_date)
        if not stockdata.empty:
            file_path = os.path.join(folder_path, f'{ticker}.csv')
            stockdata.to_csv(file_path)
            
            print(f"Saved data to {file_path} for {ticker}")
    
            total_files_downloaded += 1
            
    except (ValueError, KeyError) as e:
        print(f"Failed to fetch the data for {ticker} due to : {e}")
        error_tickers.append({"Ticker": ticker, "Error Type": str(e)})

    except requests.exceptions.ConnectionError as e:
        print(f"Failed to connect for {ticker} due to: {e}")
        error_tickers.append({"Ticker": ticker, "Error Type": "Connection Error"})

    except Exception as e:
        error_message = str(e)
        if "NameResolutionError" in error_message and "query2.finance.yahoo.com" in error_message:
            print(f"Failed to connect for {ticker} due to: {e}")
            error_tickers.append({"Ticker": ticker, "Error Type": "Connection Error (Failed to connect)"})
        else:
            print(f"Unexpected error occured for {ticker} due to : {e}")
            error_tickers.append({"Ticker": ticker, "Error Type": "Unexpected Error"})

if len(error_tickers) > 0:
    print("Tickers with errors:")
    error_df = pd.DataFrame(error_tickers)
    print(error_df)
else:
    print("No errors occurred for any tickers.")
    
print(f"Total number of files downloaded stand at : {total_files_downloaded} on {end_date}")

In [ ]:
output_base_folder = r"C:\Users\dbcin\OneDrive\Desktop\Consolidated Stock Data"
output_folder = os.path.join(output_base_folder,datetime.today().strftime('%d-%B-%Y'))

import datetime

# Get the current date and time
now = datetime.datetime.now()

# Check if the current time is before or after 12 PM
if now.hour < 12:
    today_date = (now.date() - datetime.timedelta(days=1)).strftime('%d-%m-%Y')
else:
    today_date = now.date().strftime('%d-%m-%Y')

print(f"Today's Date is = {today_date}")

output_file =  f"Consolidated Data for {today_date}.xlsx"

if not os.path.exists(output_folder):
    os.makedirs(output_folder)
    print(f"Output Folder '{output_folder}' created.")
else:
    print(f"Folder '{output_folder}' already exists.")

In [ ]:
#Consolidating Close Data

consolidated_data_close = pd.DataFrame()
consolidated_data_open = pd.DataFrame()
consolidated_data_high = pd.DataFrame()
consolidated_data_low = pd.DataFrame()
consolidated_volume = pd.DataFrame()

#Close
for filename in os.listdir(folder_path):
    if filename.endswith(".csv"):
            file_path = os.path.join(folder_path , filename)
            stock_data = pd.read_csv(file_path)
        
            close_data = stock_data[['Date','Close']]       
            close_data.set_index("Date", inplace = True)
                    
                    
            stock_symbol = os.path.splitext(filename)[0]
            close_data.rename(columns = {"Close": stock_symbol}, inplace = True)

            consolidated_data_close = pd.concat([consolidated_data_close,close_data],axis = 1)       
        
#Open
for filename in os.listdir(folder_path):
    if filename.endswith(".csv"):
            file_path = os.path.join(folder_path , filename)
            stock_data = pd.read_csv(file_path)
        
            open_data = stock_data[['Date','Open']]       
            open_data.set_index("Date", inplace = True)
                    
                    
            stock_symbol = os.path.splitext(filename)[0]
            open_data.rename(columns = {"Open": stock_symbol}, inplace = True)
        
            consolidated_data_open = pd.concat([consolidated_data_open,open_data],axis = 1)


#High
for filename in os.listdir(folder_path):
    if filename.endswith(".csv"):
            file_path = os.path.join(folder_path , filename)
            stock_data = pd.read_csv(file_path)
        
            high_data = stock_data[['Date','High']]       
            high_data.set_index("Date", inplace = True)
                    
                    
            stock_symbol = os.path.splitext(filename)[0]
            high_data.rename(columns = {"High": stock_symbol}, inplace = True)
        
            consolidated_data_high = pd.concat([consolidated_data_high,high_data],axis = 1)

#Low
for filename in os.listdir(folder_path):
    if filename.endswith(".csv"):
            file_path = os.path.join(folder_path , filename)
            stock_data = pd.read_csv(file_path)
        
            low_data = stock_data[['Date','Low']]       
            low_data.set_index("Date", inplace = True)
                    
                    
            stock_symbol = os.path.splitext(filename)[0]
            low_data.rename(columns = {"Low": stock_symbol}, inplace = True)
        
            consolidated_data_low = pd.concat([consolidated_data_low,low_data],axis = 1)


#Volume
for filename in os.listdir(folder_path):
    if filename.endswith(".csv"):
            file_path = os.path.join(folder_path , filename)
            stock_data = pd.read_csv(file_path)
        
            volume_data = stock_data[['Date','Volume']]       
            volume_data.set_index("Date", inplace = True)
                    
                    
            stock_symbol = os.path.splitext(filename)[0]
            volume_data.rename(columns = {"Volume": stock_symbol}, inplace = True)
        
            consolidated_volume = pd.concat([consolidated_volume,volume_data],axis = 1)

In [ ]:
def check_synchronization(dataframes):
    # Check column headers
    first_columns = [df.columns.tolist() for df in dataframes]
    if not all(col == first_columns[0] for col in first_columns):
        raise ValueError("Column headers are not synchronized across dataframes.")

    # Check indexes (dates)
    first_indexes = [df.index.tolist() for df in dataframes]
    if not all(idx == first_indexes[0] for idx in first_indexes):
        raise ValueError("Indexes (dates) are not synchronized across dataframes.")

dataframes = [consolidated_data_close, consolidated_data_open, consolidated_data_high, consolidated_data_low, consolidated_volume]

# Check synchronization of column headers and indexes across dataframes
try:
    check_synchronization(dataframes)
    print("Column headers and indexes are synchronized across dataframes.")
except ValueError as e:
    print(f"Error: {e}")

In [ ]:
def synchronize_dates_across_dataframes(dataframes):
    # Find the common date range across all dataframes
    common_index = None
    for df in dataframes:
        if common_index is None:
            common_index = df.index
        else:
            common_index = common_index.intersection(df.index)
    
    # Print the common date range
    if common_index is not None:
        print(f"Common date range across all dataframes: {common_index.min()} to {common_index.max()}")

        # Trim all dataframes to the common date range
        for df in dataframes:
            df.drop(df.index.difference(common_index), inplace=True)
        
        # Print message indicating synchronization and updated last date
        print(f"Synchronized dates across dataframes. Last date is now: {common_index[-1]}")
    else:
        print("No common date range found among the dataframes.")

dataframes = [consolidated_data_close, consolidated_data_open, consolidated_data_high, consolidated_data_low, consolidated_volume]

# Synchronize dates across dataframes
synchronize_dates_across_dataframes(dataframes)

In [ ]:
import datetime

# Get the current date and time
now = datetime.datetime.now()

# Check if the current time is before or after 12 PM
if now.hour < 12:
    today_date = (now.date() - datetime.timedelta(days=1)).strftime('%d-%m-%Y')
else:
    today_date = now.date().strftime('%d-%m-%Y')

print(f"Today's Date is = {today_date}")


# Function to print the common date range across all dataframes
def print_common_date_range(dataframes):
    common_index = None
    for df in dataframes:
        if common_index is None:
            common_index = df.index
        else:
            common_index = common_index.intersection(df.index)
    
    if common_index is not None:
        print(f"Common date range across all dataframes: {common_index.min()} to {common_index.max()}")
    else:
        print("No common date range found among the dataframes.")


# Function to synchronize dates across all dataframes and drop the last row if necessary
def synchronize_and_check_end_date(dataframes, end_date):
    # Find the common date range across all dataframes
    common_index = None
    for df in dataframes:
        if common_index is None:
            common_index = df.index
        else:
            common_index = common_index.intersection(df.index)
    
    # Print the common date range
    if common_index is not None:
        print(f"Common date range across all dataframes: {common_index.min()} to {common_index.max()}")

        # Trim all dataframes to the common date range
        for df in dataframes:
            df.drop(df.index.difference(common_index), inplace=True)
        
        # Check if last date matches the end date
        last_date = common_index[-1]
        if last_date != end_date:
            print(f"Last date {last_date} does not match the expected end date {end_date}. Dropping last row from all dataframes.")
            
            # Store the last row for potential reinsertion
            last_rows = {i: df.loc[last_date].copy() for i, df in enumerate(dataframes)}

            # Drop the last row from all dataframes
            for df in dataframes:
                df.drop(last_date, inplace=True)
            
            # Re-check and print updated last date
            common_index = None
            for df in dataframes:
                if common_index is None:
                    common_index = df.index
                else:
                    common_index = common_index.intersection(df.index)
            
            if common_index is not None:
                last_date = common_index[-1]
                print(f"New last date after dropping row: {last_date}")

            # Ask user if they want to reinsert the dropped rows
                user_input = input("Do you want to reinsert the dropped rows? (yes/no): ").strip().lower()
                if user_input == 'yes':
                    for i, df in enumerate(dataframes):
                        df.loc[last_rows[i].name] = last_rows[i]
                    print("Dropped rows have been reinserted.")
                else:
                    print("Chosen not to re-insert dropped rows.")
                print_common_date_range(dataframes)
            else:
                print("No common date range found among the dataframes after dropping row.")
        else:
            print(f"Last date matches the expected end date {end_date}. Proceeding with data as is.")
    else:
        print("No common date range found among the dataframes.")

    return dataframes, last_date

# Assuming these dataframes are defined somewhere before this point
dataframes = [consolidated_data_close, consolidated_data_open, consolidated_data_high, consolidated_data_low, consolidated_volume]

# Define your end date here (replace with actual end date logic)
end_date = today_date

# Synchronize dates across dataframes and check end date
dataframes, last_date = synchronize_and_check_end_date(dataframes, end_date)

In [ ]:
output_path = os.path.join(output_folder,output_file)

last_filled_date = consolidated_data_close.index[-1]

print(f"The last filled date is: {last_filled_date}")

empty_stocks = consolidated_data_close.columns[consolidated_data_close.loc[last_filled_date].isna()]

if not empty_stocks.empty:
    print(f"Number of stocks with empty values = {len(empty_stocks)}")
    print(f"Stocks with empty values on {last_filled_date}:")
    for stock_symbol in empty_stocks:
        print(stock_symbol)
else:
    print(f"No stocks with empty values on {last_filled_date}.")

#Writing the file to excel

with pd.ExcelWriter(output_path) as writer:
        consolidated_data_close.to_excel(writer, sheet_name = "Close", index=True)
        consolidated_data_open.to_excel(writer, sheet_name = "Open", index=True)
        consolidated_data_high.to_excel(writer, sheet_name = "High", index=True)
        consolidated_data_low.to_excel(writer, sheet_name = "Low", index=True)
        consolidated_volume.to_excel(writer, sheet_name = "Volume", index=True)
    
print(f"Consolidated data stored in {output_path}.\nGet to work now. ")
